# Least squares with SGD

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import urllib
from pathlib import Path
import tarfile
import pandas as pd
import seaborn as sns
import hashlib
import scipy

In [ ]:
import matplotlib as mpl
mpl.rcParams['axes.grid'] = True  # Use per default a grid, i.e. plt.grid()
# mpl.rcParams['figure.figsize'] = [6.4, 4.8]  # Change the default figure size

- `Tab`: Code completion or indent
- `Shift-Tab`: Tooltip
- `Shift-Tab Shift+Tab`: Long tooltip
- `Ctrl-Enter`: Run selected cells
- `Shift-Enter`: Run cell, select below
- `Alt-Enter`: Run cell and insert below
- `Ctrl-#`: Comment
- `Esc a`: New cell above
- `Esc b`: New cell below
- `Esc d d`: Delete current cell
- `Esc m`: Turn current cell into Markdown cell
- `Esc y`: Turn current cell into code cell

# Download the California household prices dataset

- Each row contains data about a block group (600 to 3000 people)
- Some values are missing/ clipped or scaled.

Values in the dataset:
1. longitude: A measure of how far west a house is; a higher value is farther west
2. latitude: A measure of how far north a house is; a higher value is farther north
3. housingMedianAge: Median age of a house within a block; a lower number is a newer building
4. totalRooms: Total number of rooms within a block
5. totalBedrooms: Total number of bedrooms within a block
6. population: Total number of people residing within a block
7. households: Total number of households, a group of people residing within a home unit, for a block
8. medianIncome: Median income for households within a block of houses (measured in tens of thousands of US Dollars)
9. medianHouseValue: Median house value for households within a block (measured in US Dollars)
10. oceanProximity: Location of the house w.r.t ocean/sea

Inspect the dataset and discuss what the `pairplot` shows.
Does it help to decide, which features might be good to predict `median_house_value`?

For those that use `cocalc.com`:
If you do not have internet access in your Jupyter notebook, you need to download the `housing.tgz` manually and then upload it to your environment just as you uploaded the notebook file.

In [ ]:
csv_path = Path('housing.csv')

if not csv_path.is_file():
    remote_url = 'https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz'
    local_path = Path('housing.tgz')

    if not local_path.is_file():
        urllib.request.urlretrieve(remote_url, local_path)

    with tarfile.open(local_path) as f:
        f.extractall()

In [ ]:
df = pd.read_csv(csv_path)
df.head()

In [ ]:
df.info()

In [ ]:
df["ocean_proximity"].value_counts()

In [ ]:
df.describe()

In [ ]:
# Check https://seaborn.pydata.org/examples/index.html
sns.pairplot(df.dropna(), diag_kws={'bins': 50});

# Remove missing data and clipped data

When data is missing or is corrupted, you basically have three options:
- Drop the row (i.e. the record containing the missing value)
- Drop the column (i.e. drop the complete feature)
- Fill with a sane default value (often median of the train set, has to be stored for the test conditions later)

In [ ]:
# Drops all rows which contain NaN anywhere
df = df.dropna()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(6.4 * 2, 4.8))
for ax, feature in zip(axes, ['median_house_value', 'housing_median_age']):
    ax.hist(df[feature], bins=50)
    ax.set_title(feature)
plt.show()

### Questions:

Are there corrupted values?
 
 - Do you have an idea, why they are corrupted?
 - How can you identify them?
 - Check https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html and remove them.

In [ ]:
df['median_house_value'].max(), df['housing_median_age'].max()  # REPLACE

In [ ]:
df = df.query('median_house_value < 500001')  # REPLACE df = df.query(???)
df = df.query('housing_median_age < 52')  # REPLACE df = df.query(???)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(6.4*2, 4.8))
for ax, feature in zip(axes, ['median_house_value', 'housing_median_age']):
    ax.hist(df[feature], bins=50)
    ax.set_title(feature)
plt.show()

# Split dataset into subsets

It is absolutely crutial to split the dataset into at least two subsets:
- Train set
- Test set

Often, people tend to split the dataset into three different subsets:
- Train set
- Cross-validation set/ Validation set
- Test set

You would then train on the train set and always check how your algorithm performs on the CV set.
Just for the final results you would run your model once on the test set but not change any parameters afterwards anymore.
The best model has to be selected based on the CV set.

In [ ]:
df['_id'] = df.apply(lambda row: '{}_{}'.format(
    row["longitude"],
    row["latitude"]
), axis=1)

df.head()

Now we split the dataset into a train and test set.

### Questions:

 - Do you get a random test set?
 - Is the test set each time the same set?
 - What would happen, when we get new samples or decide to drop more sample? Would the train/test assignment change?

In [ ]:
def is_in_test_set(key, test_ratio):
    hash_value = hashlib.md5(key.encode()).digest()
    return hash_value[-1] < 256 * test_ratio

def split_train_test(dataframe, test_ratio):
    in_test_set = dataframe._id.apply(lambda _id: is_in_test_set(_id, test_ratio))
    return dataframe.loc[~in_test_set], dataframe.loc[in_test_set]

df_train, df_test = split_train_test(df, 0.2)

In [ ]:
print('df:       {}'.format(len(df)))
print('df_train: {}'.format(len(df_train)))
print('df_test:  {}'.format(len(df_test)))

# Model the median house value based on median income

Fit the ordinary least squares model to predict median house values based on median income.

$y_n = w_0 + w_1 x_n + v_n$ with $v_n \sim \mathcal{N}(0, \sigma_v^2)$

### Matrix notation:
The objective can be written as $J =  \lVert\mathbf{y} - \mathbf{X}\boldsymbol{\theta}\rVert^2$ with 

 - Observation matrix: $\mathbf{X} = \begin{bmatrix} 1 & x_{1,1} & \dots & x_{1,D-1} \\ \vdots & \vdots & \ddots & \vdots \\ 1 & x_{N,1} & \dots & x_{N,D-1} \end{bmatrix} \in \mathbb{R}^{N \times (D)} = \mathbb{R}^{14744 \times 2}$
 - Target vector $\mathbf{y} = \begin{bmatrix}y_1\\\vdots\\y_N\end{bmatrix}$
 - Parameter vector $\boldsymbol{\theta} = \begin{bmatrix}w_0\\\vdots\\w_{D-1}\end{bmatrix}$
 
After some simple steps (Try it, it is important to be able to optimize such an objective) we obtain $\boldsymbol{\theta} = (\mathbf{X}^\mathrm{T} \mathbf{X})^{-1} \mathbf{X}^\mathrm{T}\mathbf{y}$

### Questions:
 - The start equation is a polynom with order 1.
   It is easy to see, that this is a linear classifier.
   When we increase the polynom order $y_n = w_0 + w_1 x_n + \dots + w_{D-1} x_n^{D-1} + v_n$, would it be still a linear classifier?
 - Why is there a column of ones in the observation matrix?
 - Does $(\mathbf{X}^\mathrm{T} \mathbf{X})^{-1}$ exist? How about $(\mathbf{X} \mathbf{X}^\mathrm{T})^{-1}$?


In [ ]:
ylabel = 'median house value in $'

y = np.asarray(df_train.median_house_value)
y.shape

Now we want to get the Observation matrix $\mathbf{X}$.
Take the vector `df_train.median_house_value` (1D-Array) and convert it to a matrix `X`, where you add a column with ones.

Hints:
 - There are several solutions.
 - One solution is based on `np.stack` and `np.ones_like`


In [ ]:
xlabel = 'median income in 10 000 $'

def get_observation_matrix(feature_vector):
    X = np.stack([np.ones_like(feature_vector), feature_vector], axis=-1)  # REPLACE X = ???
    return X

X = get_observation_matrix(df_train.median_income)
print(X[:5, :])
X.shape

Now we want to estimate the parameter vector `theta`.

Hint:
 
 - You may want to use `np.linalg.solve`, `np.linalg.inv`, `np.linalg.pinv` or `scipy.linalg.lstsq`.
 - Are some of them better than others? If yes, why?

In [ ]:
%timeit theta = np.linalg.pinv(X) @ y  # REPLACE
%timeit theta, _, _, _ = scipy.linalg.lstsq(X, y)  # REPLACE
%timeit theta = np.linalg.solve(X.T @ X, X.T @ y)  # REPLACE
theta = np.linalg.solve(X.T @ X, X.T @ y)  # REPLACE theta = ???
theta

Plot the training and test data and the regression model.

 - What do you expect?
 - Do you see, what you expected?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

def partial_plot(ax, dataframe, title, theta):
    ax.scatter(dataframe.median_income, dataframe.median_house_value, alpha=0.01)
    x_plot = np.linspace(0, 12)  # REPLACE ???
    y_plot = get_observation_matrix(x_plot) @ theta   # REPLACE
    ax.plot(x_plot, y_plot, color='red')  # REPLACE ax.plot(???, ???, color='red')
    ax.set_title(title)
    ax.set_xlim((0, 12))
    ax.set_ylim((0, 500000))
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

partial_plot(axes[0], df_train, 'Train set', theta)
partial_plot(axes[1], df_test, 'Test set', theta)

Calculate the root mean squared error of your predictions both on the train and on the test set.

In [ ]:
y_test = np.asarray(df_test.median_house_value)
y_test.shape

In [ ]:
X_test = get_observation_matrix(df_test.median_income)
X_test.shape

In [ ]:
def root_mean_squared_error(X, y, theta):# REPLACE
    return np.sqrt((y - X @ theta) @ (y - X @ theta) / X.shape[0])  # REPLACE
%timeit root_mean_squared_error(X, y, theta)  # REPLACE
# REPLACE
def root_mean_squared_error(X, y, theta):
    difference = y - X @ theta  # REPLACE difference = ???
    assert difference.shape == (X.shape[0],), 'Unexpected shape: {}'.format(difference.shape)
    return np.sqrt((difference) @ difference / X.shape[0])  # REPLACE return ???

%timeit root_mean_squared_error(X, y, theta)  # REPLACE root_mean_squared_error(X, y, theta)

In [ ]:
# Check your root_mean_squared_error implementation with a toy example
rmse = root_mean_squared_error(
    X=np.array([
        [1, 2],
        [1, 3],
        [1, 4],
    ]),
    y=np.array([10, 12, 14]),
    theta=np.array([6, 2])
)
assert rmse == 0, rmse

In [ ]:
X.shape, y.shape, theta.shape

In [ ]:
print('Train error: {:10.0f} US$'.format(
    root_mean_squared_error(X, y, theta)
))

print('Test error:  {:10.0f} US$'.format(
    root_mean_squared_error(X_test, y_test, theta)
))

# Stochastic gradient descent

Implement a (stochastic) gradient descent algorithm to estimate the linear regression parameters.
Ideally, write a function which allows to set different parameter, e.g. mini batch size.
Add some kind of learning rate schedule (i.e. learning rate decreases by a factor of 10 after each epoch after the 5th).

Hints:
 - Objective: $J = \lVert\mathbf{y} - \mathbf{X}\boldsymbol{\theta}\rVert^2$
 - Gradient: $\displaystyle\frac{\partial J}{\partial \boldsymbol{\theta}}$
 - Gradient descent algorithm: $\displaystyle\boldsymbol{\theta}^\mathrm{new} = \boldsymbol{\theta}^\mathrm{old} - \mu \left.\frac{\partial J}{\partial \boldsymbol{\theta}}\right|_{\boldsymbol{\theta} = \boldsymbol{\theta}^\mathrm{old}}$ where $\mu$ is the learning rate
 - Stochastic gradient descent algorithm: Use gradient descent algorithm with a subset of $\mathbf{X}$ (i.e. mini batch)
    - Important: Shuffle the training data. Often they are sorted and then the mini batch is far away to be representative for the complete data.
 - Epoch: One epoch means, that you processed all training data one time.

Functions you may want to use:
 - `np.random.shuffle`
 - `np.array_split`

In [ ]:
def get_learning_rate(epoch):
    # This is a fixed learning rate schedule.
    initial_learning_rate = 0.01  # REPLACE ???
    if epoch <= 5:  # REPLACE return ???
        return initial_learning_rate  # REPLACE
    else:  # REPLACE
        return initial_learning_rate * 10 ** -(epoch - 5)  # REPLACE

def get_gradient_theta(X, y, theta):
    batch_size = X.shape[0]  # REPLACE
    return -(X.T @ y - X.T @ X @ theta) / batch_size  # REPLACE return ???

def fit(
    X_train, y_train,
    X_test, y_test,
    epochs=10,
    batch_size=10,
    logging=True
):
    number_of_examples, number_of_parameters = X_train.shape
    
    # Set up batching parameters
    all_example_indices = list(range(number_of_examples))
    number_of_batches = -(-number_of_examples // batch_size)
    
    # Initialize variables
    theta = np.zeros(number_of_parameters)
    
    # Prepare logging
    if logging:
        train_error_history = []
        test_error_history = []
        theta_history = []
    else:
        train_error_history = None
        test_error_history = None
        theta_history = None
    
    for epoch in range(epochs):
        learning_rate = get_learning_rate(epoch)
        np.random.shuffle(all_example_indices)  # REPLACE ???  # shuffle all_example_indices
        
        for mini_batch in np.array_split(all_example_indices, number_of_batches):  # REPLACE ???  # for mini_batch in ???:
            gradient = get_gradient_theta(  # REPLACE ???
                X_train[mini_batch, :],  # REPLACE
                y_train[mini_batch],  # REPLACE
                theta  # REPLACE
            )  # REPLACE
            theta = theta - learning_rate * gradient  # REPLACE
            
            if logging:
                theta_history.append(theta)
                train_error_history.append(
                    root_mean_squared_error(X_train, y_train, theta)
                )
                test_error_history.append(
                    root_mean_squared_error(X_test, y_test, theta)
                )
    
    if logging:
        train_error_history = np.asarray(train_error_history)
        test_error_history = np.asarray(test_error_history)
        theta_history = np.asarray(theta_history)
    
    return theta, train_error_history, test_error_history, theta_history

theta, train_error_history, test_error_history, theta_history = fit(
    X, y,
    X_test, y_test,
    epochs=10,
    batch_size=10,
    logging=True
)
fig, axes = plt.subplots(1, 2, figsize=(16, 4))

ax = axes[0]
ax.plot(np.log(train_error_history), label='Train error')
ax.plot(np.log(test_error_history), color='red', label='Test error')
ax.set_xlabel('Iteration')
ax.set_ylabel('Root Mean Squared Error')
ax.legend()

ax = axes[1]
ax.plot(theta_history[:, 0], theta_history[:, 1])
ax.scatter(theta[0], theta[1], color='red', zorder=3, s=100)
ax.set_xlabel('w_0')
ax.set_ylabel('w_1')

plt.show()
print('Best test loss: {:10.0f}'.format(np.min(test_error_history)))

In [ ]:
%%timeit  # REPLACE
theta, train_error_history, test_error_history, theta_history = fit(  # REPLACE
    X, y,  # REPLACE
    X_test, y_test,  # REPLACE
    epochs=10,  # REPLACE
    batch_size=10,  # REPLACE
    logging=False  # REPLACE
)  # REPLACE

In [ ]:
%%timeit  # REPLACE
theta, train_error_history, test_error_history, theta_history = fit(  # REPLACE
    X, y,  # REPLACE
    X_test, y_test,  # REPLACE
    epochs=10,  # REPLACE
    batch_size=10,  # REPLACE
    logging=True  # REPLACE
)  # REPLACE

# For experts...

- Check the somewhat famous paper [1] from Yoshua Bengio. Especially Section 2 is worth a read. Also Section 3.1.1 provides useful information about learning rate schedules and other hyper-parameters.
- Implement early stopping (stop when CV loss does not decrease for a few epochs).
- Implement learning rate back-off based on CV loss.

You can also add more features from the dataset to try to predict the prices better.
How would you use the `ocean_proximity` feature.

- [1] https://arxiv.org/pdf/1206.5533.pdf